# Assignment 2: Hypothesis Testing

*Due*: Wed, Nov 13, 2019 in class

*Submission*: Complete this notebook and print out the output or electronically submit it. 

Everything you need to complete is marked with a TODO. For textual questions create a new cell under the question to respond to it.


## Motivation
In a standard randomized control trial, our null hypothesis is often trivial---nothing happens, no difference in the mean, no difference in the relative ranking. In this assignment, we look to generalize this idea to compare observed data against an assumed statistical model. That is, could the observed data plausibly be generated from the known model.


An air shower is a cascade of ionized particles and electromagnetic radiation produced in the atmosphere when a primary cosmic ray (i.e. one of extraterrestrial origin) enters the atmosphere. When a particle, which could be a proton, a nucleus, an electron, a photon, or (rarely) a positron, strikes an atom's nucleus in the air it produces many energetic hadrons. We have a detector that observes particles that reach a ground station and measures the particle energy and arrival time.

## Model and Data
We have the following theoretical model to describe particle behavior. The energy of each particle is drawn independent of arrival time from a [Gamma Distribution](https://en.wikipedia.org/wiki/Gamma_distribution). The particles arrive as a [Poisson process](https://en.wikipedia.org/wiki/Poisson_point_process). We have the following simulator:

In [3]:
import numpy as np
import pandas as pd

def simulate_burst(total):
    """Simulates a trial of total # of particles returns
       a dataframe with two columns one with observed time (in microsecs)
       and one with the energy in kilojoules.
    """
    t = 0
    data = []
    for trial in range(total):
        t += np.random.exponential(scale=1.0)
        obs = np.random.gamma(2.15,1.96, 1)[0]
        
        data.append({'otime_us':t, 'energy_kj':obs})
    
    return pd.DataFrame(data)

simulate_burst(100)

,otime_us,energy_kj
0,0.685612,5.178483
1,0.952404,5.741685
2,1.502043,1.680501
3,1.540914,3.771643
4,4.275446,3.646512
...,...,...
95,91.236333,5.672314
96,92.086414,3.486321
97,93.856540,5.960679
98,95.092457,0.857877


In addition to the simulator, you are given a daset of real observations [download](https://raw.githubusercontent.com/sjyk/cmsc21800/master/part.csv). You will write a function to load this dataset into a pandas dataframe. The dataset contains some missing values, the function should drop all rows with any missing values (i.e., NaN)

In [4]:
def load_data(filename):
    """Input: a csv file of airburst observations
       Output: a pandas dataframe with no NaNs
    """
    
    df = pd.read_csv(filename, delimiter = ' ')#note the delimeter is a space
    return df[~df['energy_kj'].isna()]

df = load_data('part.csv')
df

,otime_us,energy_kj
0,0.940633,4.897721
1,1.559228,-128.000000
2,2.222809,0.122285
3,3.144400,9.691012
4,4.805632,2.356866
...,...,...
994,994.849265,3.151195
995,995.726746,2.719565
996,996.395580,0.153446
997,997.955123,3.958193


## Pre-Processing

Before, we begin testing, we show analyze the data for potential problems.

Q1. TODO *Compare particle energies generated from the simulator and the real data. If they do differ, explain how.*

In [5]:
simulate_burst(1000).describe()

,otime_us,energy_kj
count,1000.000000,1000.000000
mean,506.560277,4.247356
std,289.401569,2.871127
min,0.541664,0.066098
25%,266.499774,2.207631
50%,513.919807,3.554632
75%,741.870036,5.750241
max,1031.537303,17.050303


In [6]:
df.describe()

,otime_us,energy_kj
count,996.000000,996.000000
mean,498.045568,4.385513
std,287.746567,33.995733
min,0.940633,-246.000000
25%,249.466874,1.940490
50%,497.730739,3.559943
75%,746.774461,5.958558
max,998.868280,286.000000


Q2. TODO *Your engineers tell you that all energy readings should be positive. Are there any negative values in either of the datasets? If so, is there any unexpected pattern to those values in terms of times they occur or values they take on?*

In [9]:
df_orig = load_data('part.csv') 
df = df_orig[df_orig['energy_kj'] > 0]
df = load_data('part.csv') 

Q3. TODO *Are there any other energy readings that are suspect in the real dataset? Roughly what fraction of values are suspect*

In [25]:
#Code for Q3

Based on your answers to Q1, Q2, Q3, write a function that cleans the real data by removing all problematic observations. 

In [10]:
def clean(df):
    """
       Input: a dataframe of a mix of erroneous and correct observations 
       Output: a dataframe with only the correct observations
    """
    
    return df[(df['energy_kj'] > 0) & (df['energy_kj'] < 20)]

## Comparing the Energies
Now, we will compare the particle energies from the simulated data and the real data. Fill in the following hypothesis tests. *Be reasonable about this* You may not import methods from statistics packages that perform the test for you.

In [16]:
from scipy.stats import *
import numpy as np

def z_test_energy(simulated, real):
    """Input: a dataframe of simulated observations, a dataframe of real observations 
       Output: a p-value based on the two-sample z-test
    """
    
    diff = np.abs(np.mean(simulated['energy_kj']) - np.mean(real['energy_kj']))
    
    se = np.sqrt(sem(simulated['energy_kj'])**2 + sem(real['energy_kj'])**2 )
    
    return (1-norm.cdf(diff/se))*2, (1-norm.cdf(diff/se))


#get the data
simulated = simulate_burst(10000)
real =  clean(load_data('part.csv'))
print('Two-Sample Z-Test: ', z_test_energy(simulated, real))

Two-Sample Z-Test:  0.30617660754744647


There are a few parameters here that we selected: (1) how much data to simulate, and (2) we cleaned the real dataset prior to testing. Evaluate the effects of these choices.

Q1. TODO *Would it be beneficial to simulate 1e6 data points rather than the 10000 used above, why or why not?*

In [ ]:
#Code for Q1

Q2. TODO *Do the p-values change if you did not clean the dataset? Are both tests equally sensitive to the dirty data?*

In [ ]:
#Code for Q2

## Comparing the Arrival Times
So far, we have only tested the particle energies. Another important aspect of our model is the arrival process (i.e., the times). 

Q1. TODO *Describe a hypothesis test that evaluates whether the arrival process significantly differs in the simulator from the observed data.*

Q2. TODO *Do your pre-processing choice above change? Why or why not?*

In [ ]:
def test_arrival_process(simulated, real):
    """TODO
    
       Input: a dataframe of simulated observations, a dataframe of real observations 
       Output: a p-value that determines the difference between the arrival processes
    """
    
    raise ValueError("NotImplemented: You must implement rs_test_energy")